In [ ]:
import pandas as pd
import numpy as np
import joblib
from collections import Counter
from joblib import load

# --------------------
# 1. Charger les modèles
# --------------------
encoder = load(r"C:\Users\fgrol\Documents\stages\models\label_encoder.pkl")
knn_model = joblib.load(r"C:\Users\fgrol\Documents\stages\models\knn_model.pkl")
rf_model = joblib.load(r"C:\Users\fgrol\Documents\stages\models\rf_model.pkl")
catboost_model = joblib.load(r"C:\Users\fgrol\Documents\stages\models\catboost_model.pkl")
svm_model = joblib.load(r"C:\Users\fgrol\Documents\stages\models\svm_model.pkl")

# --------------------
# 2. Colonnes features
# --------------------
colonnes_features = [
    'x', 'y', 'altitude', 'hauteur', 'vitesse',
    'vitesse_calculee', 'delta_t', 'vz',
    'distance', 'temps_total', 'distance_totale',
    'longitude_depart'
]
# --------------------
# 3. Charger fichier CSV
# --------------------
fichier = r"C:\Users\fgrol\Documents\stages\csv_test.csv"
df = pd.read_csv(fichier, sep=',', quotechar='"', engine='python', nrows=10000)


# --------------------
# 4. Nettoyer DataFrame
# --------------------
X = df[colonnes_features].apply(pd.to_numeric, errors='coerce')
X = X.replace([np.inf, -np.inf], np.nan)
print("NaNs par colonne avant suppression :")
print(X.isna().sum())
X = X.dropna()
print(f"Lignes après nettoyage: {len(X)}")

# --------------------
# 5. Prédictions
# --------------------
pred_knn = encoder.inverse_transform(knn_model.predict(X))
pred_rf = encoder.inverse_transform(rf_model.predict(X))
pred_cb = encoder.inverse_transform(catboost_model.predict(X))
pred_svm = encoder.inverse_transform(svm_model.predict(X))

# --------------------
# 6. Ajouter prédictions au DataFrame
# --------------------
df_predictions = df.loc[X.index].copy()
df_predictions['KNN'] = pred_knn
df_predictions['RandomForest'] = pred_rf
df_predictions['CatBoost'] = pred_cb
df_predictions['SVM'] = pred_svm

# --------------------
# 7. Calcul vote majoritaire
# --------------------
def vote_majoritaire(row):
    votes = [row['KNN'], row['RandomForest'], row['CatBoost'], row['SVM']]
    vote_counts = Counter(votes)
    return vote_counts.most_common(1)[0][0]

df_predictions['majorité'] = df_predictions.apply(vote_majoritaire, axis=1)

# --------------------
# 8. Calcul % de justesse
# --------------------
def accuracy(y_true, y_pred):
    return (y_true == y_pred).mean() * 100

from IPython.display import Markdown

# Calcul des scores
accuracy_dict = {
    'KNN': accuracy(df_predictions['modele_standardise'], df_predictions['KNN']),
    'Random Forest': accuracy(df_predictions['modele_standardise'], df_predictions['RandomForest']),
    'CatBoost': accuracy(df_predictions['modele_standardise'], df_predictions['CatBoost']),
    'SVM': accuracy(df_predictions['modele_standardise'], df_predictions['SVM']),
    'Vote majoritaire': accuracy(df_predictions['modele_standardise'], df_predictions['majorité']),
}

# Création d’un tableau pour l’affichage
df_scores = pd.DataFrame.from_dict(accuracy_dict, orient='index', columns=['Taux de réussite (%)'])
df_scores.index.name = 'Modèle'
df_scores = df_scores.round(2)

display(Markdown("### 🎯 **Taux de justesse par modèle**"))
display(df_scores.style.set_properties(**{'text-align': 'center'}))



# --------------------
# 9. Comparaison majorité vs vérité par drone puis par trajectoire (Voila-ready)
# --------------------

from IPython.display import display
import pandas as pd

# 1. Fonction pour récupérer la valeur majoritaire
def mode_majoritaire(series):
    return series.mode().iloc[0] if not series.mode().empty else 'Inconnu'

# 2. Grouper par drone_id puis traj_id
grouped = df_predictions.groupby(['drone_id', 'traj_id'])

# 3. Aggrégation
comparaison_par_traj = grouped.agg({
    'modele_standardise': mode_majoritaire,
    'majorité': mode_majoritaire
}).reset_index()

# 4. Ajout d'une colonne 'correct'
comparaison_par_traj['correct'] = comparaison_par_traj['modele_standardise'] == comparaison_par_traj['majorité']

# 5. Format final pour Voilà
df_voila = comparaison_par_traj.rename(columns={
    'drone_id': 'Drone ID',
    'traj_id': 'Trajectoire ID',
    'modele_standardise': 'Modèle réel',
    'majorité': 'Prédiction majorité',
    'correct': 'Correct'
})

# 6. Trier pour meilleure lisibilité (optionnel)
df_voila = df_voila.sort_values(by=['Drone ID', 'Trajectoire ID']).reset_index(drop=True)

# 7. Ajout d’un style pour l’affichage via Voilà
def highlight_correct(val):
    if val is True:
        return 'background-color: #d4edda; color: black;'  # vert pâle
    elif val is False:
        return 'background-color: #f8d7da; color: black;'  # rouge pâle
    return ''

styled_df = df_voila.style.applymap(highlight_correct, subset=['Correct']) \
                          .set_properties(**{'text-align': 'center'}) \
                          .set_table_styles([dict(selector='th', props=[('text-align', 'center')])])

# 8. Affichage final pour Voilà
display(styled_df)



# --------------------
# 10. Exporter résultats
# --------------------
colonnes_export = ['modele_standardise', 'KNN', 'RandomForest', 'CatBoost', 'SVM', 'majorité']
df_predictions[colonnes_export].to_csv(r"C:\Users\fgrol\Documents\stages\algo\csv_predictions_output2.csv", index=False)
print("\n✅ Exporté avec vote majoritaire dans : csv_predictions_output.csv")

from IPython.display import HTML

display(HTML(f"""
<div style="background-color:#e2f0d9; padding:10px; border-radius:8px; margin-top:10px;">
✅ <strong>Résultats exportés</strong> dans : <code>csv_predictions_output2.csv</code><br>
📁 Lignes initiales : <strong>{len(df)}</strong><br>
🧹 Lignes après nettoyage : <strong>{len(X)}</strong>
</div>
"""))



## 📁 Veuillez charger un fichier CSV :

FileUpload(value=(), accept='.csv', description='Upload')

Button(button_style='success', description="Lancer l'analyse", style=ButtonStyle())

Output()

In [3]:
print(f"Nombre de lignes : {len(df)}")


Nombre de lignes : 1000
